In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

In [ ]:
sns.set_context('talk')

# Load data

In [ ]:
df = pd.read_csv('data/train.csv').set_index('customer')

In [ ]:
del_columns = ['category', 'nationality', 'is_pep']
for var_ in del_columns:
    df.drop(var_, axis=1, inplace=True)

In [ ]:
df['suspicious'].astype(int).sum()

In [ ]:
df.head()

# Basic statistics

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.boxplot(df['age'])

# Create balanced dataset

In [ ]:
cases_susp = df[df['suspicious']==1]
cases_norm = df[df['suspicious']==0].sample(n=cases_susp.shape[0])
print(cases_susp.shape, cases_norm.shape)

In [ ]:
df_bal = pd.concat([cases_norm, cases_susp])
df_bal.shape

# Model

In [ ]:
test_features = pd.read_csv('data/test.csv').set_index('customer')

In [ ]:
for var_ in del_columns:
    test_features.drop(var_, axis=1, inplace=True)

In [ ]:
test_features.shape

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Train

In [ ]:
sub = df_bal  #.sample(n=100)

In [ ]:
X = sub.drop('suspicious', axis=1)
y = sub['suspicious']

In [ ]:
X.shape

#### Create pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
clf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(n_estimators=1000, n_jobs=2, verbose=1))
])

#### Splitting

In [ ]:
skf = StratifiedKFold(n_splits=2)

In [ ]:
%%time
for train_index, test_index in tqdm(skf.split(X, y), total=skf.get_n_splits(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred)
    
    print(report)

#### Final

In [ ]:
%%time
clf.fit(X, y)

### Test

In [ ]:
sub_test = test_features  # [['nationality', 'atm_withdrawal']]
sub_test.shape

In [ ]:
%%time
predictions = clf.predict_proba(sub_test)

In [ ]:
# sort by `suspicious` probability
df_pred = pd.DataFrame(
    predictions,
    columns=['normal_prob', 'suspicious_prob'],
    index=sub_test.index
).sort_values(by='suspicious_prob', ascending=False)
df_pred.head()

In [ ]:
N = 1000
idx = df_pred['suspicious_prob'].head(N).index
fraud_rows = test_features.loc[idx]

In [ ]:
(pd.Series(fraud_rows.index)
 .to_frame()
 .to_csv('fraudulent_customers.txt', index=False))

In [ ]:
sns.distplot(df_pred['suspicious_prob'], kde=False)
plt.ylabel('Count')